In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
conn = psycopg2.connect("dbname=fleebmarket user=fleebmarket host=localhost")

In [96]:
cur = conn.cursor()


In [97]:
cur.execute("SELECT author, c from (select author, count(*) as c from search_app_redditadvert where ad_type = 'Selling' group by author) as T where c > 2;")
data = cur.fetchall()


In [98]:
authors_df = pd.DataFrame(data, columns=["author", "count"])
authors_df

,author,count
0,captinfapin,6
1,savageastr0naut,8
2,faagtfook,3
3,TieuNgu,8
4,H3NT4I,3
...,...,...
2801,bigislittle,5
2802,HavocHybrid,8
2803,jeefixD,3
2804,domasleo,4


In [100]:
authors_list = ','.join([
    f"'{author}'" for author in authors_df['author']
])
cur.execute(f"SELECT author, extra -> 'offers', created_utc, reddit_id from search_app_redditadvert  where author in ({authors_list}) and ad_type = 'Selling' order by created_utc desc;")
data = cur.fetchall()
df = pd.DataFrame(data, columns=['author', 'offers', 'date', 'reddit_id'])
df

,author,offers,date,reddit_id
0,CityAtSpeed,"JWK Moss Switches, Greetech Browns, Free PBT Islander Fix Kit",2022-01-31 23:43:14+00:00,shg1h2
1,liceaben,"GMK Sloth novelties, S-Craft Artisan",2022-01-31 23:17:09+00:00,shffun
2,hpfontenot,"Fully assembled Mode65, Mode65 PC Back, FR4 Plate for KBD67 Lite, Kiwi switches x140",2022-01-31 22:29:50+00:00,shea4r
3,PapaD1gg1ty,"Ultramarine Fjell, Black Alu KBD67 Lite R2",2022-01-31 22:27:25+00:00,she7vm
4,Kasiio_,"Switches, Lubed, Stock, Tecsee, Cherry, Gateron",2022-01-31 22:21:53+00:00,she30h
...,...,...,...,...
18917,fartnaround,KBD fans 5 degree with DZ60rgb hot swappable,2021-05-10 14:35:00+01:00,n94ku2
18918,oli4731,Tofu WKL Black w/ Koalas L&F,2021-05-10 14:18:04+01:00,n948bp
18919,ds2465,Grey 7v + Extras,2021-05-10 14:03:46+01:00,n93y07
18920,siwankim,Maple story Artisan keycaps: White yeti,2021-05-10 13:41:07+01:00,n93hof


In [27]:
pd.set_option('display.max_colwidth', None)

In [110]:
import re
from fuzzywuzzy import fuzz
import logging


MIN_SIMILAR_GROUPS = 1
MIN_SIMILAR_WORDS = 8
MIN_FUZZ_SIMILARITY = 90


def tokenize_groups(terms):
    tokens = re.split(r",|\|| / | // ", terms)
    return [
        token.strip(' ').lower()
        for token in tokens
        if len(token.strip(' ')) > 0
    ]

def similar_groups(terms1, terms2):
    tokens1 = tokenize_groups(terms1)
    tokens2 = tokenize_groups(terms2)
    return len(set(tokens1) & set(tokens2))

def tokenize_words(terms):
    return [
        t.lower()
        for t in terms.split()
        if len(t) > 2
    ]

def similar_words(terms1, terms2):
    tokens1 = tokenize_words(terms1)
    tokens2 = tokenize_words(terms2)
    return len(set(tokens1) & set(tokens2))


def similar_words_debug(terms1, terms2):
    tokens1 = tokenize_words(terms1)
    tokens2 = tokenize_words(terms2)
    return set(tokens1) & set(tokens2)

def similar_fuzz(terms1, terms2):
    return fuzz.partial_ratio(terms1, terms2)


In [115]:

res = {
    "group": 0,
    "words": 0,
    "fuzz" : 0
}

fuzz_only = []
group_only = []
words_only = []

for i, row in df.head(500).iterrows():
    dups_candidates = df[(df['author'] == row['author']) & (df['date'] < row['date'] )]
    for i, irow in dups_candidates.iterrows():
        is_group = False
        is_words = False
        is_fuzz = False
        if similar_groups(row["offers"], irow["offers"]) >= MIN_SIMILAR_GROUPS:
            res["group"] += 1
            is_group = True
        if similar_words(row["offers"], irow["offers"]) >= MIN_SIMILAR_WORDS:
            res['words'] += 1
            is_words = True
        if similar_fuzz(row["offers"], irow["offers"]) >= MIN_FUZZ_SIMILARITY:
            res['fuzz'] += 1
            is_fuzz = True
        if is_fuzz and not (is_group or is_words):
            fuzz_only.append((row["offers"], irow["offers"], row['reddit_id'], irow['reddit_id']))
        if is_group and not (is_fuzz or is_words):
            group_only.append((row["offers"], irow["offers"]))
        if is_words and not (is_fuzz or is_group):
            words_only.append((row["offers"], irow["offers"], row['reddit_id'], irow['reddit_id']))
res

{'group': 1378, 'words': 425, 'fuzz': 514}

In [116]:

len(fuzz_only), \
len(group_only), \
len(words_only)

(92, 669, 19)

In [117]:
words_only

[('Iron 180 WKL e-white unbuilt, GMK DMG base + spacebar + ZMKC artisan',
  'Iron 180 WKL white alum unbuilt, Sam S7.7 ano silver hotswap, GMK DMG base + ZMKC artisan, GMK Boneyard base + 40s kits',
  'sgplwi',
  's8aos6'),
 ('Built Dark Grey Meridian Bundled w/ Matching Keycaps + Flemo Cable',
  'Built Dark Grey Meridian Bundled w/ Keycaps + Flemo Cable',
  'sggvt2',
  'sdi874'),
 ('Rama Thermal kuro WK hotswap built with black ink v2 and durock stabs | Geon Frog Harbor Gray F12 WK H87c mill max | GMK WOB | Drop SA genespeed | modo 2 aesthetic | modo 2 alternate',
  'sealed GMK WOB | Rama Thermal kuro winkey beam hotswap | Rama Thermal HHKB kuro hotswap beam | MT3 biip extended 2048 alpha+mod+accent mod | MT3 dev/tty | Geon F1-8x red | MODO L aesthetic sealed',
  'sgd4xw',
  's4ubkd'),
 ('SodieCaps Death Wish Pilgrim , Cysm pink Keyby metal keycap bundle',
  'MONOKEI x CYSM bundle keyby artisan keycap , Nightcaps Bombking , Death wish pilgrim sodiecaps',
  'sfxo72',
  'sdotsz'),
 ('An

In [109]:
similar_groups('Ansi Novatouch with og bke domes, Zenith ZKB-2 with PBT caps from 163-73 and SKCM Blues, Goupil(cherry) g81-3061haf, NEC pc-9801-98-s02',
  'Ascii Askeyboard sono1(skcm browns)+sono2(skcm ambers), Zenith ZKB-2 with pbt caps from Zenith 163-73 and skcm blues, NOS Wang 725 skcm oranges')

0

In [118]:
similar_words_debug('Keychron Q1 w/ custom mechcables aviator cable/ Akko Neon Laser Keycaps, Built KBD67 Lite R2 with Akko Green/ Shark PBT Keycaps',
  'Built KBD67 Lite with PBT Taro, Keyspresso RBG hotswap numpad, Mechcables purple coiled aviator cable, tai hao dark tunnel keycaps')

{'aviator', 'built', 'kbd67', 'keycaps', 'lite', 'mechcables', 'pbt', 'with'}

In [ ]:
Zenith ZKB-2 with PBT caps from 163-73 and SKCM Blues
Zenith ZKB-2 with pbt caps from Zenith 163-73 and skcm blues